<a href="https://colab.research.google.com/github/kurehajime/openapi_sandbox/blob/main/%E9%8C%B2%E9%9F%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_js(filename='record.mp3'):
  js = Javascript('''
    async function record() {
      let rec;
      let chanks;


      // HTML組み立て
      const div = document.createElement('div');
      const startRecord = document.createElement('button');
      startRecord.textContent = 'Rec';
      div.appendChild(startRecord);

      const stopRecord = document.createElement('button');
      stopRecord.textContent = 'Stop';
      stopRecord.style.display = 'none'
      div.appendChild(stopRecord);

      const message = document.createElement('div');
      div.appendChild(message);

      const audio = document.createElement('audio');
      div.appendChild(audio);

      document.body.appendChild(div);

      // Audioが有効になったら
      function handlerFunction(stream,resolve) {
          message.textContent = 'handlerFunction';
          rec = new MediaRecorder(stream);
          // 録音が完了したら
          rec.ondataavailable = e => {
              message.textContent = 'ondataavailable';
              chanks.push(e.data);
              if (rec.state == "inactive") {
                  let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
                  audio.src = URL.createObjectURL(blob);
                  audio.controls = true;
                  audio.autoplay = true;
                  resolve();
              }
          }
      }

      startRecord.onclick = e => {
          message.textContent = 'start';
          startRecord.style.display = 'none'
          stopRecord.style.display = 'block'
          chanks = [];
          rec.start();
      }

      stopRecord.onclick = e => {
        message.textContent = 'stop';
        startRecord.style.display = 'block'
        stopRecord.style.display = 'none'
        rec.stop();
      }

      function blobToBase64(blob) {
        return new Promise((resolve, _) => {
          const reader = new FileReader();
          reader.onloadend = () => resolve(reader.result);
          reader.readAsDataURL(blob);
        });
      }
      
      await new Promise((resolve) => {
        navigator.mediaDevices.getUserMedia({ audio: true })
            .then(stream => { handlerFunction(stream,resolve) })
      });
      let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
      return await blobToBase64(blob);
    }
    ''')
  display(js)
  data = eval_js('record()')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [120]:
try:
  filename = record_js()
except Exception as err:
  print(str(err))

<IPython.core.display.Javascript object>